In [1]:
# Importing files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import sys
import glob
import datetime
import calendar
import csv

# Spark imports
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.functions import col, asc, desc, split

from pyspark.sql.functions import array_contains # used to check if the array has a value

In [2]:
# Initialize a spark session.
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [3]:
spark = init_spark() # Initializate spark

In [4]:
# BUSINESS.JSON

In [5]:
# Importing Business.json file
path = 'data/yelp_academic_dataset_business.json'
df_business = spark.read.json(path)

In [6]:
# Print the total number of rows and columns
print((df_business.count(), len(df_business.columns)))

(150346, 14)


In [7]:
# Print the columns
df_business.columns

['address',
 'attributes',
 'business_id',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'name',
 'postal_code',
 'review_count',
 'stars',
 'state']

In [8]:
# Print the dtypes of each column
df_business.dtypes

[('address', 'string'),
 ('attributes',
  'struct<AcceptsInsurance:string,AgesAllowed:string,Alcohol:string,Ambience:string,BYOB:string,BYOBCorkage:string,BestNights:string,BikeParking:string,BusinessAcceptsBitcoin:string,BusinessAcceptsCreditCards:string,BusinessParking:string,ByAppointmentOnly:string,Caters:string,CoatCheck:string,Corkage:string,DietaryRestrictions:string,DogsAllowed:string,DriveThru:string,GoodForDancing:string,GoodForKids:string,GoodForMeal:string,HairSpecializesIn:string,HappyHour:string,HasTV:string,Music:string,NoiseLevel:string,Open24Hours:string,OutdoorSeating:string,RestaurantsAttire:string,RestaurantsCounterService:string,RestaurantsDelivery:string,RestaurantsGoodForGroups:string,RestaurantsPriceRange2:string,RestaurantsReservations:string,RestaurantsTableService:string,RestaurantsTakeOut:string,Smoking:string,WheelchairAccessible:string,WiFi:string>'),
 ('business_id', 'string'),
 ('categories', 'string'),
 ('city', 'string'),
 ('hours',
  'struct<Friday:st

In [9]:
# Printing how many Null files each categorie have
for i in df_business.columns:
    print(i, df_business.filter(col(i).isNull()).count())

address 0
attributes 13744
business_id 0
categories 103
city 0
hours 23223
is_open 0
latitude 0
longitude 0
name 0
postal_code 0
review_count 0
stars 0
state 0


In [10]:
# Printing the categories column
df_business.select("categories").show(25, False) # Printing the categories in the dataset for the first 25 instances

# ----------------- Comments --------------
# 1 - Each row is given as a single string

+----------------------------------------------------------------------------------------------------------+
|categories                                                                                                |
+----------------------------------------------------------------------------------------------------------+
|Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists|
|Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services                            |
|Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores                        |
|Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries                                                     |
|Brewpubs, Breweries, Food                                                                                 |
|Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants                              |
|Sporting Goods, Fa

In [11]:
# Printing the categories in the dataset for the first 25 instances
df_business.select("attributes").show(10, False) 

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|attributes                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
+---------------

In [12]:
# Filtering the rows if the categories contains RESTAURANT
df_business_filtered = df_business.filter(df_business.categories.contains('Restaurants'))

In [13]:
# Print the number of rows and columns of the dataframe filtered
print((df_business_filtered.count(), len(df_business_filtered.columns)))

(52268, 14)


In [14]:
# Printing how many Null files each categorie have after filtering
for i in df_business.columns:
    print(i, df_business_filtered.filter(col(i).isNull()).count())

address 0
attributes 565
business_id 0
categories 0
city 0
hours 7278
is_open 0
latitude 0
longitude 0
name 0
postal_code 0
review_count 0
stars 0
state 0


In [15]:
# Apllying one-hot encoding to categories feature

import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

df = df_business_filtered.toPandas()
mlb = MultiLabelBinarizer()
df['categories'] = df['categories'].str.split(',')

df_categories = pd.DataFrame(mlb.fit_transform(df['categories']),columns=mlb.classes_, index=df.index)
df_categories

,Acai Bowls,Accessories,Accountants,Active Life,Acupuncture,Adult,Adult Entertainment,Advertising,Afghan,African,...,Wholesale Stores,Wholesalers,Wigs,Wine Bars,Wine Tasting Room,Wine Tours,Wineries,Women's Clothing,Wraps,Yoga
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Counting the categories
categories_counts = df['categories'].value_counts()
categories_counts

[Restaurants,  Pizza]                                                                                                                                                                                                                935
[Pizza,  Restaurants]                                                                                                                                                                                                                823
[Restaurants,  Mexican]                                                                                                                                                                                                              728
[Restaurants,  Chinese]                                                                                                                                                                                                              708
[Mexican,  Restaurants]                                             

In [17]:
# Pre-processing the feature City
# Changing all letter to lowercase

df = df.sort_values(by=['city'])  # Sorting by name
print('Original unique city values:', len(df['city'].unique()))  # Printing the unique values
df['city'] = df['city'].str.lower()
print('Unique city values after changing all letter to lowercase:', len(df['city'].unique())) # Printing the unique values

Original unique city values: 920
Unique city values after changing all letter to lowercase: 859


In [18]:
# Encoding feature City and State features

df_city = pd.get_dummies(df.city, prefix='City')
df_state = pd.get_dummies(df.state, prefix='State')
df_state # printing for state (example)

,State_AB,State_AZ,State_CA,State_CO,State_DE,State_FL,State_HI,State_ID,State_IL,State_IN,State_LA,State_MO,State_MT,State_NC,State_NJ,State_NV,State_PA,State_TN,State_XMS
25839,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
42911,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
33631,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6695,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9078,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3171,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
48502,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
25977,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
43985,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4260,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [19]:
# # Counting the categories
categories_counts = df['city'].value_counts()
categories_counts

philadelphia               5854
tampa                      2967
indianapolis               2862
nashville                  2504
tucson                     2469
new orleans                2261
edmonton                   2168
saint louis                1790
reno                       1288
boise                       848
santa barbara               767
clearwater                  679
wilmington                  620
st. louis                   544
metairie                    522
saint petersburg            491
franklin                    444
st. petersburg              405
sparks                      334
brandon                     327
largo                       312
meridian                    312
cherry hill                 297
carmel                      291
west chester                281
kenner                      228
new port richey             227
goleta                      220
palm harbor                 208
greenwood                   207
                           ... 
bywater 

In [20]:
###############
## USER FILE ##
###############

In [21]:
# Loading the user file
path = 'data/yelp_academic_dataset_user.json'
df_user = spark.read.json(path)

In [22]:
# Print the number of rows and columns
print((df_user.count(), len(df_user.columns)))

(1987897, 22)


In [23]:
#Printing the columns
df_user.columns

['average_stars',
 'compliment_cool',
 'compliment_cute',
 'compliment_funny',
 'compliment_hot',
 'compliment_list',
 'compliment_more',
 'compliment_note',
 'compliment_photos',
 'compliment_plain',
 'compliment_profile',
 'compliment_writer',
 'cool',
 'elite',
 'fans',
 'friends',
 'funny',
 'name',
 'review_count',
 'useful',
 'user_id',
 'yelping_since']

In [24]:
# Printing how many Null files each categorie have
for i in df_user.columns:
    print(i, df_user.filter(col(i).isNull()).count())

average_stars 0
compliment_cool 0
compliment_cute 0
compliment_funny 0
compliment_hot 0
compliment_list 0
compliment_more 0
compliment_note 0
compliment_photos 0
compliment_plain 0
compliment_profile 0
compliment_writer 0
cool 0
elite 0
fans 0
friends 0
funny 0
name 0
review_count 0
useful 0
user_id 0
yelping_since 0


In [35]:
df_user_filtered = df_user.select('user_id', 'review_count')

In [38]:
df_user_filtered.show(10)

+--------------------+------------+
|             user_id|review_count|
+--------------------+------------+
|qVc8ODYU5SZjKXVBg...|         585|
|j14WgRoU_-2ZE1aw1...|        4333|
|2WnXYQFK0hXEoTxPt...|         665|
|SZDeASXq7o05mMNLs...|         224|
|hA5lMy-EnncsH4JoR...|          79|
|q_QQ5kBBwlCcbL1s4...|        1221|
|cxuxXkcihfCbqt5By...|          12|
|E9kcWJdJUHuTKfQur...|         358|
|lO1iq-f75hnPNZkTy...|          40|
|AUi8MPWJ0mLkMfwbu...|         109|
+--------------------+------------+
only showing top 10 rows



In [40]:
df_user_filtered = df_user_filtered.filter(col('review_count')>=10)

In [42]:
# Printing the number of users with more than 10 reviews
df_user_filtered.count()

726519

In [25]:
#################
## REVIEW FILE ##
#################

In [26]:
path = 'data/yelp_academic_dataset_review.json'
df_review = spark.read.json(path)

In [27]:
# Print the number of rows and columns
print((df_review.count(), len(df_review.columns)))

(6990280, 9)


In [28]:
#Printing the columns
df_review.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

In [29]:
# Printing how many Null files each categorie have
for i in df_review.columns:
    print(i, df_review.filter(col(i).isNull()).count())

business_id 0
cool 0
date 0
funny 0
review_id 0
stars 0
text 0
useful 0
user_id 0


In [62]:
df_review_filtered = df_review.select('business_id', 'review_id', 'stars','user_id')
df_review_filtered.show(10)

+--------------------+--------------------+-----+--------------------+
|         business_id|           review_id|stars|             user_id|
+--------------------+--------------------+-----+--------------------+
|XQfwVwDr-v0ZS3_Cb...|KU_O5udG6zpxOg-Vc...|  3.0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|BiTunyQ73aT9WBnpR...|  5.0|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|saUsX_uimxRlCVr67...|  3.0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|AqPFMleE6RsU23_au...|  5.0|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|Sx8TMOWLNuJBWer-0...|  4.0|bcjbaE6dDog4jkNY9...|
|04UD14gamNjLY0IDY...|JrIxlS1TzJ-iCu79u...|  1.0|eUta8W_HdHMXPzLBB...|
|gmjsEdUsKpj9Xxu6p...|6AxgBCNX_PNTOxmbR...|  5.0|r3zeYsv1XFBRA4dJp...|
|LHSTtnW3YHCeUkRDG...|_ZeMknuYdlQcUqng_...|  5.0|yfFzsLmaWF2d4Sr0U...|
|B5XSoSG3SfvQGtKEG...|ZKvDG2sBvHVdF5oBN...|  3.0|wSTuiTk-sKNdcFypr...|
|gebiRewfieSdtt17P...|pUycOfUwM8vqX7KjR...|  3.0|59MxRhNVhU9MYndMk...|
|uMvVYRgGNXf5boolA...|rGQRf8UafX7OTlMNN...|  5.0|1WHRWwQmZOZDAhp2Q...|
|EQ-TZ

In [66]:
# Filtering REVIEWS by only users identified with >= 20 REVIEWS
df_review_filtered2 = df_review_filtered.join(df_user_filtered, df_review_filtered.user_id == df_user_filtered.user_id ,"left_semi")
df_review_filtered2.count()

4863844

In [71]:
# Filtering REVIEWS file by only RESTAURANTS identified with the restaurant CATEGORIES
df_review_filtered3 = df_review_filtered2.join(df_business_filtered[['business_id']], df_review_filtered2.business_id == df_business_filtered.business_id ,"left_semi")
df_review_filtered3.count()

3432479

In [106]:
df_review_filtered4 = df_review_filtered3[['user_id','business_id']].distinct()

In [107]:
df_review_filtered4.count()

3290578

In [ ]:
df_review_filtered3 = df_review_filtered3[['user_id','business_id']].distinct()

In [116]:
df_review_filtered3[['user_id']].show(20, truncate=False)

+----------------------+
|user_id               |
+----------------------+
|--0DrQkM0FT-yCQRWw82uQ|
|--1orhUoGFSdHXsoxqQc8g|
|--1orhUoGFSdHXsoxqQc8g|
|--3Hl2oAvTPlq-f7KtogJg|
|--3Hl2oAvTPlq-f7KtogJg|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4AjktZiHowEIBCMd4CZA|
|--4T2OChSFzSvcYG8-lQPA|
+----------------------+
only showing top 20 rows



In [100]:
i1 = np.array([[1, 8, 6],
      [-2, 5, 9],
      [1, 1, 1]])

u = np.array([1, 3, 5])

np.matmul(u,i1)/np.sum(u)

array([0.        , 3.11111111, 4.22222222])

In [117]:
path = 'data/yelp_academic_dataset_review.json'
df_review1 = spark.read.json(path).limit(20)

In [122]:
df_review1 = df_review1[['business_id','user_id','stars']]

In [123]:
df_review1.show(truncate=False)

+----------------------+----------------------+-----+
|business_id           |user_id               |stars|
+----------------------+----------------------+-----+
|XQfwVwDr-v0ZS3_CbbE5Xw|mh_-eMZ6K5RLWhZyISBhwA|3.0  |
|7ATYjTIgM3jUlt4UM3IypQ|OyoGAe7OKpv6SyGZT5g77Q|5.0  |
|YjUWPpI6HXG530lwP-fb2A|8g_iMtfSiwikVnbP2etR0A|3.0  |
|kxX2SOes4o-D3ZQBkiMRfA|_7bHUi9Uuf5__HHc_Q8guQ|5.0  |
|e4Vwtrqf-wpJfwesgvdgxQ|bcjbaE6dDog4jkNY91ncLQ|4.0  |
|04UD14gamNjLY0IDYVhHJg|eUta8W_HdHMXPzLBBZhL1A|1.0  |
|gmjsEdUsKpj9Xxu6pdjH0g|r3zeYsv1XFBRA4dJpL78cw|5.0  |
|LHSTtnW3YHCeUkRDGyJOyw|yfFzsLmaWF2d4Sr0UNbBgg|5.0  |
|B5XSoSG3SfvQGtKEGQ1tSQ|wSTuiTk-sKNdcFyprzZAjg|3.0  |
|gebiRewfieSdtt17PTW6Zg|59MxRhNVhU9MYndMkz0wtw|3.0  |
|uMvVYRgGNXf5boolA9HXTw|1WHRWwQmZOZDAhp2Qyny4g|5.0  |
|EQ-TZ2eeD_E0BHuvoaeG5Q|ZbqSHbgCjzVAqaa7NKWn5A|4.0  |
|lj-E32x9_FA7GmUrBGBEWg|9OAtfnWag-ajVxRbUTGIyg|4.0  |
|RZtGWDLCAtuipwaZ-UfjmQ|smOvOajNG0lS4Pq7d8g4JQ|4.0  |
|otQS34_MymijPTdNBoBdCw|4Uh27DgGzsp6PqrH913giQ|4.0  |
|BVndHaLihEYbr76Z0CMEGw|1C2l

In [135]:
dt_temp = df_review1.filter(col('user_id') == 'mh_-eMZ6K5RLWhZyISBhwA')

In [138]:
dt_temp.show()

+--------------------+--------------------+-----+
|         business_id|             user_id|stars|
+--------------------+--------------------+-----+
|XQfwVwDr-v0ZS3_Cb...|mh_-eMZ6K5RLWhZyI...|  3.0|
+--------------------+--------------------+-----+



In [145]:
df_temp2 = df_business_filtered[['business_id','city']].join(dt_temp, ['business_id'], how="full_outer")

In [230]:
np_stars = np.array(df_temp2.select('stars').collect())

In [165]:
np_items_matrix = np.array(df_business_filtered.collect())

In [167]:
df_business = pd.read_json('data\yelp_academic_dataset_business.json',lines=True)
df_business

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,"1616 Chapala St, Ste 2",{'ByAppointmentOnly': 'True'},Pns2l4eNsfO8kk83dixA6A,"Doctors, Traditional Chinese Medicine, Naturop...",Santa Barbara,None,0,34.426679,-119.711197,"Abby Rappoport, LAC, CMQ",93101,7,5.0,CA
1,87 Grasso Plaza Shopping Center,{'BusinessAcceptsCreditCards': 'True'},mpf3x-BjTdTEA3yCZrAYPw,"Shipping Centers, Local Services, Notaries, Ma...",Affton,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",1,38.551126,-90.335695,The UPS Store,63123,15,3.0,MO
2,5255 E Broadway Blvd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",tUFrWirKiKi_TAnsVWINQQ,"Department Stores, Shopping, Fashion, Home & G...",Tucson,"{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",0,32.223236,-110.880452,Target,85711,22,3.5,AZ
3,935 Race St,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",MTSW4McQd7CbVtyjqoe9mw,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Philadelphia,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",1,39.955505,-75.155564,St Honore Pastries,19107,80,4.0,PA
4,101 Walnut St,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...",mWMc6_wTdE0EUBKIGXDVfA,"Brewpubs, Breweries, Food",Green Lane,"{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",1,40.338183,-75.471659,Perkiomen Valley Brewery,18054,13,4.5,PA
5,615 S Main St,"{'BusinessParking': 'None', 'BusinessAcceptsCr...",CF33F8-E6oudUQ46HnavjQ,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",Ashland City,"{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",1,36.269593,-87.058943,Sonic Drive-In,37015,6,2.0,TN
6,"8522 Eager Road, Dierbergs Brentwood Point","{'BusinessAcceptsCreditCards': 'True', 'Restau...",n_0UpQx1hsNbnPUSlodU8w,"Sporting Goods, Fashion, Shoe Stores, Shopping...",Brentwood,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",1,38.627695,-90.340465,Famous Footwear,63144,13,2.5,MO
7,400 Pasadena Ave S,None,qkRM_2X51Yqxk3btlwAQIg,"Synagogues, Religious Organizations",St. Petersburg,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,27.766590,-82.732983,Temple Beth-El,33707,5,3.5,FL
8,8025 Mackenzie Rd,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...",k0hlBqXX-Bt0vf1op7Jr1w,"Pubs, Restaurants, Italian, Bars, American (Tr...",Affton,None,0,38.565165,-90.321087,Tsevi's Pub And Grill,63123,19,3.0,MO
9,2312 Dickerson Pike,"{'RestaurantsAttire': ''casual'', 'Restaurants...",bBDDEgkFA1Otx9Lfe7BZUQ,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",Nashville,"{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...",1,36.208102,-86.768170,Sonic Drive-In,37207,10,1.5,TN


In [168]:
df_business = df_business[df_business['categories'].str.contains('Restaurants',na=False)]
df_business

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
3,935 Race St,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",MTSW4McQd7CbVtyjqoe9mw,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Philadelphia,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",1,39.955505,-75.155564,St Honore Pastries,19107,80,4.0,PA
5,615 S Main St,"{'BusinessParking': 'None', 'BusinessAcceptsCr...",CF33F8-E6oudUQ46HnavjQ,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",Ashland City,"{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",1,36.269593,-87.058943,Sonic Drive-In,37015,6,2.0,TN
8,8025 Mackenzie Rd,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...",k0hlBqXX-Bt0vf1op7Jr1w,"Pubs, Restaurants, Italian, Bars, American (Tr...",Affton,None,0,38.565165,-90.321087,Tsevi's Pub And Grill,63123,19,3.0,MO
9,2312 Dickerson Pike,"{'RestaurantsAttire': ''casual'', 'Restaurants...",bBDDEgkFA1Otx9Lfe7BZUQ,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",Nashville,"{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...",1,36.208102,-86.768170,Sonic Drive-In,37207,10,1.5,TN
11,,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...",eEOYSgkmpB90uNA7lDOMRA,"Vietnamese, Food, Restaurants, Food Trucks",Tampa Bay,"{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",1,27.955269,-82.456320,Vietnamese Food Truck,33602,10,4.0,FL
12,8901 US 31 S,"{'RestaurantsReservations': 'False', 'Restaura...",il_Ro8jwPlHresjw9EGmBg,"American (Traditional), Restaurants, Diners, B...",Indianapolis,"{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ...",1,39.637133,-86.127217,Denny's,46227,28,2.5,IN
14,2575 E Bay Dr,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...",0bPLkL0QhhPO5kt1_EXmNQ,"Food, Delis, Italian, Bakeries, Restaurants",Largo,"{'Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'...",0,27.916116,-82.760461,Zio's Italian Market,33771,100,4.5,FL
15,205 Race St,"{'RestaurantsReservations': 'True', 'Restauran...",MUTTqe8uqyMdBl186RmNeA,"Sushi Bars, Restaurants, Japanese",Philadelphia,"{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-...",1,39.953949,-75.143226,Tuna Bar,19106,245,4.0,PA
19,1224 South St,"{'NoiseLevel': 'u'quiet'', 'GoodForMeal': '{'d...",ROeacJQwBeh05Rqg7F6TCg,"Korean, Restaurants",Philadelphia,"{'Monday': '11:30-20:30', 'Tuesday': '11:30-20...",1,39.943223,-75.162568,BAP,19147,205,4.5,PA
20,10359 104 Street NW,"{'OutdoorSeating': 'False', 'Caters': 'True', ...",WKMJwqnfZKsAae75RMP6jA,"Coffee & Tea, Food, Cafes, Bars, Wine Bars, Re...",Edmonton,"{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ...",0,53.546045,-113.499169,Roast Coffeehouse and Wine Bar,T5J 1B9,40,4.0,AB


In [169]:
mlb = MultiLabelBinarizer()

#create boolean mask matched non NaNs values
mask = df_business['categories'].notnull()

#filter by boolean indexing
arr = mlb.fit_transform(df_business.loc[mask, 'categories'].dropna().str.strip('[]').str.split(','))

#create DataFrame and add missing (NaN)s index values
df_categories = (pd.DataFrame(arr, index=df_business.index[mask], columns=mlb.classes_)
               .reindex(df_business.index, fill_value=0))

df_categories.sort_index()

,Acai Bowls,Accessories,Accountants,Active Life,Acupuncture,Adult,Adult Entertainment,Advertising,Afghan,African,...,Wholesale Stores,Wholesalers,Wigs,Wine Bars,Wine Tasting Room,Wine Tours,Wineries,Women's Clothing,Wraps,Yoga
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [170]:
df_business = df_business.sort_values(by=['city'])  # Sorting by name
print('Original unique city values:', len(df_business['city'].unique()))  # Printing the unique values
df_business['city'] = df_business['city'].str.lower()
print('Unique city values after changing all letter to lowercase:', len(df_business['city'].unique())) # Printing the unique values

Original unique city values: 920
Unique city values after changing all letter to lowercase: 859


In [171]:
# Encoding feature City and State features
df_city = pd.get_dummies(df_business.city).sort_index()
df_state = pd.get_dummies(df_business.state).sort_index()
df_city # printing for state (example)

,abington,abington township,affton,aldan,algiers,alloway,alton,ambler,antioch,apollo beach,...,wycombe,wyncote,wyndlake condominium,wyndmoor,wynnewood,yardley,yeadon,zephyrhills,zieglerville,zionsville
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [172]:
df_restData = df_business[['business_id', 'latitude','longitude','stars','is_open']].sort_index()

In [173]:
frames = [df_restData, df_categories, df_city, df_state]
df_itemProf = pd.concat(frames, axis=1)
df_itemProf.sort_index()

,business_id,latitude,longitude,stars,is_open,Acai Bowls,Accessories,Accountants,Active Life,Acupuncture,...,IN,LA,MO,MT,NC,NJ,NV,PA,TN,XMS
3,MTSW4McQd7CbVtyjqoe9mw,39.955505,-75.155564,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,CF33F8-E6oudUQ46HnavjQ,36.269593,-87.058943,2.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,k0hlBqXX-Bt0vf1op7Jr1w,38.565165,-90.321087,3.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,bBDDEgkFA1Otx9Lfe7BZUQ,36.208102,-86.768170,1.5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11,eEOYSgkmpB90uNA7lDOMRA,27.955269,-82.456320,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,il_Ro8jwPlHresjw9EGmBg,39.637133,-86.127217,2.5,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
14,0bPLkL0QhhPO5kt1_EXmNQ,27.916116,-82.760461,4.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,MUTTqe8uqyMdBl186RmNeA,39.953949,-75.143226,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
19,ROeacJQwBeh05Rqg7F6TCg,39.943223,-75.162568,4.5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
20,WKMJwqnfZKsAae75RMP6jA,53.546045,-113.499169,4.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [174]:
# apply normalization technique to Column 1
df_itemProf_normalized = df_itemProf.copy()
df_itemProf_normalized['latitude'] = (df_itemProf['latitude'] - df_itemProf['latitude'].mean()) / df_itemProf['latitude'].std()    
df_itemProf_normalized['longitude'] = (df_itemProf['longitude'] - df_itemProf['longitude'].mean()) / df_itemProf['longitude'].std()      
# view normalized data  
display(df_itemProf_normalized.sort_index())

,business_id,latitude,longitude,stars,is_open,Acai Bowls,Accessories,Accountants,Active Life,Acupuncture,...,IN,LA,MO,MT,NC,NJ,NV,PA,TN,XMS
3,MTSW4McQd7CbVtyjqoe9mw,0.492047,0.918595,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,CF33F8-E6oudUQ46HnavjQ,-0.121110,0.056730,2.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,k0hlBqXX-Bt0vf1op7Jr1w,0.260762,-0.179465,3.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,bBDDEgkFA1Otx9Lfe7BZUQ,-0.131339,0.077784,1.5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11,eEOYSgkmpB90uNA7lDOMRA,-1.504208,0.389983,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,il_Ro8jwPlHresjw9EGmBg,0.439085,0.124192,2.5,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
14,0bPLkL0QhhPO5kt1_EXmNQ,-1.510721,0.367962,4.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,MUTTqe8uqyMdBl186RmNeA,0.491788,0.919488,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
19,ROeacJQwBeh05Rqg7F6TCg,0.490003,0.918087,4.5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
20,WKMJwqnfZKsAae75RMP6jA,2.752850,-1.857675,4.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [175]:
df_itemProf_normalized = df_itemProf_normalized.drop('business_id', axis = 1) #dropping business ID

In [207]:
np_items_matrix = np.array(df_itemProf_normalized)

In [208]:
np.shape(np_items_matrix)

(52268, 2008)

In [209]:
np.shape(np_stars)

(52268, 1)

In [231]:
np_stars = np_stars.astype(str)
np_stars[np_stars == 'None'] = 0
np_stars = np_stars.astype(float)

In [232]:
np.shape(np_stars.transpose())

(1, 52268)

In [233]:
result = np.matmul(np_stars.transpose(),np_items_matrix)

In [234]:
np.shape(result)

(1, 2008)

In [235]:
result

array([[ 1.56839433,  2.76281023, 10.5       , ...,  3.        ,
         0.        ,  0.        ]])